In [10]:
import json
import subprocess
from pathlib import Path
import random

import numpy as np
from xopen import xopen

# raw

In [3]:
PALOMA_DIR = "../scratch/paloma_c4_100_domains_val"

In [4]:
paloma_files = list(Path(PALOMA_DIR).iterdir())

In [5]:
obj_list = []
with xopen("../scratch/paloma_c4_100_domains_val/val_34_github.com.jsonl.gz", "r") as in_file:
    for line in in_file:
        obj = json.loads(line)
        obj_list.append(obj)

In [11]:
obj = obj_list[random.randrange(len(obj_list))]

In [12]:
obj.keys()

dict_keys(['text', 'id', 'added', 'source', 'subdomain'])

In [14]:
obj["text"]

'GOGCCFLAGS="-fPIC -m64 -pthread -fno-caret-diagnostics -Qunused-arguments -fmessage-length=0 -fdebug-prefix-map=/tmp/go-build741696436=/tmp/go-build -gno-record-gcc-switches"\ngo modules downloaded and binary is built.\nthen go install works as expected.\nI can confirm that this is happening to me as well. It is not 100% constant. My docker builds experience it about 98% of the time.\nA git fetch seems to fix it.\nSame issues with go 1.11.5, 1.11.8, 1.12.3 in Docker (golang:1.12.3), and installed locally on Ubuntu.\nReproduced locally with the same error.\nPerhaps Gerrit is acting up again?\nI\'ve traced this to a sequence of Git commands that reproduces this issue outside of Go. The Gerrit team has been notified and is working to resolve the issue.\nIs it some workaround for the problem?\nThe latest we have from the Gerrit team is that this might be a bug specific to the content of the google.golang.org/api repository. As suggested above, a git fetch should work around it. I think fe

In [20]:
obj["id"]

'bb93185a-cdc7-42a6-8195-df9ebeafd360'

In [21]:
obj["added"]

'2023-04-01T00:16:27.393489+00:00'

In [22]:
obj["source"]

'c4_100_domains'

In [23]:
obj["subdomain"]

'100_www.ign.com'

# tokenized

In [25]:
PALOMA_BIN = "../scratch/paloma_c4_100_domains_val_tokenized.bin"

In [26]:
data = np.memmap(PALOMA_BIN, dtype=np.uint16, mode="r")

In [29]:
f"{len(data):_}"

'9_761_402'

# replicate

In [35]:
cmd = (
    f"python -m lb.scripts.extract_paloma "
    f"--paloma_dir ../scratch/paloma_c4_100_domains_val "
    f"--text_dir ../scratch/text/paloma"
)
subprocess.run(cmd, shell=True)

paloma: 100%|████████████████████████| 99/99 [00:02<00:00, 40.56it/s]


CompletedProcess(args='python -m lb.scripts.extract_paloma --paloma_dir ../scratch/paloma_c4_100_domains_val --text_dir ../scratch/text/paloma', returncode=0)

In [54]:
cmd = (
    f"python -m lb.scripts.tokenize "
    f"--text_dir ../scratch/text/paloma "
    f"--tokens_file ../scratch/tokenized/paloma.bin"
)
subprocess.run(cmd, shell=True)

text_files: 100%|█████████████| 14059/14059 [00:32<00:00, 429.68it/s]


CompletedProcess(args='python -m lb.scripts.tokenize --text_dir ../scratch/text/paloma --tokens_file ../scratch/tokenized/paloma.bin', returncode=0)

In [61]:
arr_want = np.memmap(
    "../scratch/paloma_c4_100_domains_val_tokenized.bin", dtype=np.uint16, mode="r"
)

In [62]:
arr_have = np.memmap("../scratch/tokenized/paloma.bin", dtype=np.uint16, mode="r")

In [63]:
len(arr_want)

9761402

In [64]:
len(arr_have)

9761402

In [65]:
arr_want

memmap([ 3666,  3656,   290, ...,   767,    13, 50256], dtype=uint16)

In [66]:
arr_have

memmap([15905,   447,   247, ...,  4621,    13, 50256], dtype=uint16)

In [68]:
arr_want.sum()

44563429315

In [69]:
arr_have.sum()

44563429315

^ equivalent upto ordering of documents